In [ ]:
%load_ext autoreload

In [ ]:
%autoreload
# import libraries
import numpy as np
import pickle as pk
import pandas as pd
import math
# pd.options.mode.copy_on_write = True # to avoid SettingWithCopyWarning
import os
import yaml
import sys
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from pytorch_lightning.core import LightningModule
import pytorch_lightning as pl
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


%matplotlib inline

In [ ]:
#
# Read the config
#
config_path = './config/default.yaml'  
with open(config_path, 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

config = config['model_params']
print(config)

In [ ]:
class CharDataset(Dataset):
    """
    Emits batches of characters
    """
    def __init__(self, config, data):
        self.config = config
        chars = sorted(list(set(data))) + ["[MASK]"] # + mask token
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))

        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.vocab_size = vocab_size
        self.data = data

    def get_vocab_size(self):
        return self.vocab_size

    def get_block_size(self):
        return self.config['block_size']

    def __len__(self):
        return len(self.data) - self.config['block_size']

    def __getitem__(self, idx):
        return None, None
        # # grab a chunk of (block_size) characters from the data
        # chunk = self.data[idx:idx + self.config['block_size']]
        # # encode every character to an integer
        # dix = torch.tensor([self.stoi[s] for s in chunk], dtype=torch.long)

        # n_pred = max(1, int(round(self.config['block_size']*self.config['mask_prob'])))

        # masked_idx = torch.randperm(self.config['block_size'], dtype=torch.long, )[:n_pred]

        # mask = torch.zeros_like(dix)
        # mask[masked_idx] = dix[masked_idx]
        
        # dix[masked_idx] = self.stoi["[MASK]"]

        return dix, mask # Return batch + mask


In [ ]:
text = open('./minBERT/input.txt', 'r').read() 
train_dataset = CharDataset(config, text)
print(train_dataset.__len__())
dix, mask = train_dataset.__getitem__(0)
print(dix)
print()
print(mask)


In [ ]:

# setup the dataloader
# train_loader = DataLoader(
#     train_dataset,
#     sampler=torch.utils.data.RandomSampler(train_dataset, replacement=True, num_samples=int(1e10)),
#     shuffle=False,
#     pin_memory=True,
#     batch_size=config['batch_size'],
#     num_workers=config['num_workers'],
# )

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=5, 
                          pin_memory=True)

In [ ]:
data_iter = iter(train_loader)


In [ ]:
data_iter = iter(train_loader)

# fetch the next batch (x, y) and re-init iterator if needed
# try:
batch = next(data_iter)
# except StopIteration:
#     data_iter = iter(train_loader)
#     batch = next(data_iter)
# batch = [t.to(self.device) for t in batch]
# x, mask = batch
